In [ ]:
! pip install opencv-python==4.5.5.64
! pip install colab-xterm
%load_ext colabxterm
%xterm

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

In [4]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor
from loader_util.preprocessing import AspectAwarePreprocessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from imutils import paths
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

ImportError: cannot import name 'FCHeadNet' from 'loader_util.nn.conv' (C:\Users\mhasa\GitHub\deeplearning\loader_util\nn\conv\__init__.py)

In [3]:
# script constants
dataset_path = r""
model_save_dir = r""
batch_size = 32
num_epochs = 25

In [ ]:
# construct the imagegenerator
aug = ImageDataGenerator(rotation_range=30,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [ ]:
print(f"[INFO] loading images......")
image_paths = list(paths.list_images(dataset_path))
all_class_names = [pth.split(os.path.sep)[-2] for pth in image_paths]
class_names = np.unique(all_class_names)

In [ ]:
# init the image processors
aap = AspectAwarePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
data, labels = sdl.load(image_paths=image_paths, verbose=500)
data = data.astype("float") / 255.0

In [ ]:
trainx, testx, trainy, testy = train_test_split(data, labels,
                                                test_size=0.25,
                                                random_state=42)

# encode the labels
lb = LabelBinarizer()
trainy_encoded = lb.fit_transform(trainy)
testy_encoded = lb.transform(testy)

In [ ]:
# initilise the base model
base_model = VGG16(weights="imagenet",
                   include_top=False,
                   input_tensor=Input(shape=(224, 224, 3)))

head_model = FCHeadNet.build(base_model=base_model,
                             output_classes=len(class_names),
                             dense_layer_nodes=[256])
model = Model(inputs=base_model.input, outputs=head_model)

In [ ]:
# freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# now train the model
print(f"[INFO] compiling the model......")
opt = RMSprop(learning_rate=0.001)
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [ ]:
# training the head for a few epochs
print(f"[INFO] training the head......")
H = model.fit(aug.flow(trainx, trainy, batch_size=batch_size),
              validation_data=(testx, testy),
              epochs=num_epochs,
              steps_per_epoch=len(trainx) // batch_size,
              verbose=1)

In [ ]:
# evaluate the model
print("[INFO] evaluating network...")
predictions = model.predict(testx, batch_size=batch_size)
print(classification_report(testy.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=class_names))

# plot the performance
epochs = range(1, num_epochs + 1)
loss = H.history['loss']
accuracy = H.history['accuracy']
val_loss = H.history['val_loss']
val_accuracy = H.history['val_accuracy']
plot_df = pd.DataFrame(data=np.c_[epochs, loss, accuracy, val_loss, val_accuracy],
                       columns=['epochs', 'loss', 'accuracy', 'val_loss', 'val_accuracy'])

# do the actual plots
sns.set(font_scale=1)
f, ax = plt.subplots(1, 1, figsize=(15, 8))
sns.lineplot(data=plot_df, x='epochs', y='loss', ax=ax, label='train loss', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='accuracy', ax=ax, label='train accuracy', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='val_loss', ax=ax, label='val loss', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='val_accuracy', ax=ax, label='val_accuracy', linewidth=3)
ax.set_ylabel('Loss or Accuracy')
ax.set_xlabel('Epochs')
plt.setp(ax.get_legend().get_texts(), fontsize='18');  # for legend text

In [ ]:
# now unfreeze the last few layers
for layer in base_model.layers[15:]
    layer.trainable = True

In [ ]:
# now train the model
print(f"[INFO] re-compiling the model......")
opt = SGD(learning_rate=0.001)
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [ ]:
# training the head for a few epochs
print(f"[INFO] fine tuning the model......")
H = model.fit(aug.flow(trainx, trainy, batch_size=batch_size),
              validation_data=(testx, testy),
              epochs=100,
              steps_per_epoch=len(trainx) // batch_size,
              verbose=1)

In [ ]:
# evaluate the model
print("[INFO] evaluating network after finetuning...")
predictions = model.predict(testx, batch_size=batch_size)
print(classification_report(testy.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=class_names))

# plot the performance
epochs = range(1, 100 + 1)
loss = H.history['loss']
accuracy = H.history['accuracy']
val_loss = H.history['val_loss']
val_accuracy = H.history['val_accuracy']
plot_df = pd.DataFrame(data=np.c_[epochs, loss, accuracy, val_loss, val_accuracy],
                       columns=['epochs', 'loss', 'accuracy', 'val_loss', 'val_accuracy'])

# do the actual plots
sns.set(font_scale=1)
f, ax = plt.subplots(1, 1, figsize=(15, 8))
sns.lineplot(data=plot_df, x='epochs', y='loss', ax=ax, label='train loss', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='accuracy', ax=ax, label='train accuracy', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='val_loss', ax=ax, label='val loss', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='val_accuracy', ax=ax, label='val_accuracy', linewidth=3)
ax.set_ylabel('Loss or Accuracy')
ax.set_xlabel('Epochs')
plt.setp(ax.get_legend().get_texts(), fontsize='18');